## Step 1 : Extract relevant endings from the pgns

In [1]:
import subprocess
import os
import chess
import chess.pgn

cql_bin_dir = '/Users/elliottmacneil/chess/cql6/cql'
cql_scripts_dir = '/Users/elliottmacneil/python/reti/src/100endings'
output_dir = '/Users/elliottmacneil/python/reti/output/'
iterated = False
db_dir = '/Users/elliottmacneil/chess/pgn/lichess/lichess_VinayVekaria_2022-04-07.pgn'

def run_cql_scripts(cql_scripts_dir, db, count, matchlow=1):
	if os.path.isdir(cql_scripts_dir):
		for file in os.listdir(cql_scripts_dir):
			file_dir = '{}/{}'.format(cql_scripts_dir,file)
			file_noext = os.path.splitext(file)[0]
			dir_noext = os.path.join(output_dir,file_noext)
			pgn_file = dir_noext+str(count)+'.pgn'
			cql_command = '{} -i {} -o {} -matchcount {} 100 -threads 20 {}'.format(cql_bin_dir, db, pgn_file, matchlow, file_dir)
			try:
				subprocess.run(cql_command,shell=True)
			except:
				print('failed')
			count += 1
		print('Done.')
	else:
		file_noext = cql_scripts_dir.split('/')[-1][:-4]
		pgn_file = file_noext+str(count)+'.pgn'
		cql_command = '{} -i {} -o {} -matchcount {} 100 -threads 20 {}'.format(cql_bin_dir, db, pgn_file, matchlow, cql_scripts_dir)
		try:
			subprocess.run(cql_command,shell=True)
		except:
			print('failed')
		print('Done.')

In [2]:
# run each script in cql_scripts_dir on each pgn in db_dir
if iterated:
    count = 0
    for filename in os.listdir(db_dir):
        if filename.endswith('.pgn'):
            print(filename)
            db = db_dir + '/' + filename 
            run_cql_scripts(cql_scripts_dir,db,count)
            count += 1
else:
    run_cql_scripts(cql_scripts_dir,db_dir,count=1)

CQL version 6.1 (build 9.362) (c) Gady Costeff and Lewis Stiller
.........[10000]
.....
3 matches of 15352 games in 1.24 seconds using 20 threads to /Users/elliottmacneil/python/reti/output/99Qrp1.pgn
CQL version 6.1 (build 9.362) (c) Gady Costeff and Lewis Stiller
.........[10000]
.....
14 matches of 15352 games in 1.26 seconds using 20 threads to /Users/elliottmacneil/python/reti/output/77KPP2.pgn
CQL version 6.1 (build 9.362) (c) Gady Costeff and Lewis Stiller
.........[10000]
.....
13 matches of 15352 games in 1.19 seconds using 20 threads to /Users/elliottmacneil/python/reti/output/11Np3.pgn
CQL version 6.1 (build 9.362) (c) Gady Costeff and Lewis Stiller
.........[10000]
.....
175 matches of 15352 games in 1.29 seconds using 20 threads to /Users/elliottmacneil/python/reti/output/52_68RPr4.pgn
CQL version 6.1 (build 9.362) (c) Gady Costeff and Lewis Stiller
.........[10000]
.....
2 matches of 15352 games in 1.23 seconds using 20 threads to /Users/elliottmacneil/python/reti/output/

## Step 2 : Compute statistics on the pgns

In [16]:
import pandas as pd
from reti import utilities

def get_ending_stats(output_dir, original_db_dir):
    pgn = utilities.load_pgn(original_db_dir)
    db_len = utilities.alt_num_games(pgn)
    ngames = []
    prop = []
    file = []
    for filename in os.listdir(output_dir):
        if filename.endswith('.pgn'):
            file.append(filename)
            pgn_file = output_dir + '/' + filename
            pgn = utilities.load_pgn(pgn_file)
            num_games = utilities.alt_num_games(pgn)
            ngames.append(num_games)
            if db_len == 0:
                proportion = 0
            else:
                proportion = num_games/db_len*100
            prop.append(proportion)
    df = pd.DataFrame({'ending type':file,'num_games':ngames,'proportion':prop})
    csv_dir = output_dir + '/{}ending_stats.csv'.format(db_dir.split('/')[-1])
    df.to_csv(csv_dir)

get_ending_stats(output_dir, db_dir)